In [6]:
#Calcular NF, NC e ND
matriculas = [738483, 737163, 724735]
soma_matriculas = sum(matriculas)

NF = soma_matriculas % 3
NC = soma_matriculas % 2
ND = (soma_matriculas % 4) // 2

In [14]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import xgboost as xgb
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder

# Importar funções de manipulação de imagens
from operations.image_manipulation import (
    convert_image_to_gray_scale, 
    generate_image_histogram, 
    haralick_gray_scale, 
    calculate_hu_moments
)

# Função para extrair características da imagem com base no valor de NF
def extrair_caracteristicas(image_path, nf):
    # Abre a imagem
    image = Image.open(image_path)
    if nf == 0:
        # Extrair histograma 2D
        histograms = generate_image_histogram(image)
        # Se histograms for uma tupla, concatenar os histogramas
        return np.concatenate(histograms) if isinstance(histograms, tuple) else histograms
    elif nf == 1:
        # Extrair descritores de Haralick
        features = haralick_gray_scale(image)
        # Converter os descritores de Haralick em um array numpy (excluindo a imagem original)
        return np.array(list(features.values())[:-1])
    else:
        # Extrair momentos de Hu
        _, hu_moments = calculate_hu_moments(image_path, 'gray')
        # Flatten (achatar) os momentos de Hu em um vetor 1D
        return hu_moments.flatten()


In [8]:
# Diretório contendo as imagens organizadas por classe
diretorio = r'C:\Users\thayr\OneDrive\Área de Trabalho\PAI\cropped_dataset\cropped_dataset'
# Lista das classes (subpastas dentro do diretório)
classes = os.listdir(diretorio)
imagens = []
rotulos = []

# Para cada classe, carregar as imagens e extrair características
for classe in classes:
    caminho_classe = os.path.join(diretorio, classe)
    for arquivo in os.listdir(caminho_classe):
        image_path = os.path.join(caminho_classe, arquivo)
        # Extrair características da imagem
        caracteristicas = extrair_caracteristicas(image_path, NF)
        # Adicionar as características e o rótulo da classe às listas
        imagens.append(caracteristicas)
        rotulos.append(classe)

# Converter listas para arrays numpy
X = np.array(imagens)
y = np.array(rotulos)


In [9]:
# Dividir os dados em conjuntos de treino (80%) e teste (20%) com estratificação
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [18]:
# Classe negativa para o classificador binário
classe_negativa = 'ASC-US' 
# Criar rótulos binários para treino e teste (classe negativa x outras)
y_train_binario = np.where(y_train == classe_negativa, classe_negativa, 'outras')
y_test_binario = np.where(y_test == classe_negativa, classe_negativa, 'outras')

# Codificar rótulos binários como valores numéricos
label_encoder_binario = LabelEncoder()
y_train_binario_encoded = label_encoder_binario.fit_transform(y_train_binario) # Ajustando o codificador aos dados e, em seguida, transformando esses dados em valores numéricos
y_test_binario_encoded = label_encoder_binario.transform(y_test_binario) # Transformando os rótulos em valores numéricos

# Codificar rótulos multiclasse como valores numéricos
label_encoder_multiclasse = LabelEncoder()
y_train_encoded = label_encoder_multiclasse.fit_transform(y_train)
y_test_encoded = label_encoder_multiclasse.transform(y_test)


print("Usando classificador: XGBoost")
# Usar XGBoost para classificação
model_binario = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
model_multiclasse = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')

# Treinar e avaliar o classificador binário
model_binario.fit(X_train, y_train_binario_encoded) # Treinando o modelo com os dados de treinamento e seus respectivos rotulos
y_pred_binario = model_binario.predict(X_test)

# Avaliar o classificador binário
accuracy_binario = accuracy_score(y_test_binario_encoded, y_pred_binario) # Calculando a acurácia do classificador binário
conf_matrix_binario = confusion_matrix(y_test_binario_encoded, y_pred_binario) # Calculando a matriz de confusão do classificador binário
print(f"Acurácia Binário: {accuracy_binario * 100:.2f}%")
print("Matriz de Confusão Binário:\n", conf_matrix_binario)
print("Relatório de Classificação Binário:\n", classification_report(y_test_binario_encoded, y_pred_binario))

# Treinar e avaliar o classificador multiclasse
model_multiclasse.fit(X_train, y_train_encoded)
y_pred_multiclasse = model_multiclasse.predict(X_test)

# Avaliar o classificador multiclasse
accuracy_multiclasse = accuracy_score(y_test_encoded, y_pred_multiclasse)
conf_matrix_multiclasse = confusion_matrix(y_test_encoded, y_pred_multiclasse)
print(f"Acurácia Multiclasse: {accuracy_multiclasse * 100:.2f}%")
print("Matriz de Confusão Multiclasse:\n", conf_matrix_multiclasse)
print("Relatório de Classificação Multiclasse:\n", classification_report(y_test_encoded, y_pred_multiclasse))


Usando classificador: XGBoost
Acurácia Binário: 83.89%
Matriz de Confusão Binário:
 [[891  37]
 [143  46]]
Relatório de Classificação Binário:
               precision    recall  f1-score   support

           0       0.86      0.96      0.91       928
           1       0.55      0.24      0.34       189

    accuracy                           0.84      1117
   macro avg       0.71      0.60      0.62      1117
weighted avg       0.81      0.84      0.81      1117

Acurácia Multiclasse: 83.80%
Matriz de Confusão Multiclasse:
 [[  0  14   1   1   1]
 [  1 911  11   4   1]
 [  1  29  23   0   4]
 [  0  95   3   1   0]
 [  0   8   7   0   1]]
Relatório de Classificação Multiclasse:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00        17
           1       0.86      0.98      0.92       928
           2       0.51      0.40      0.45        57
           3       0.17      0.01      0.02        99
           4       0.14      0.06      